In [1]:
import numpy as np
import os
import time
from scipy.stats import moment
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Add, Input, Activation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from random import seed
from matplotlib import pyplot as plt
seed(0)

#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices()[1])

#from keras import backend as K
#K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


## Importar datos

In [2]:
#Format data:
#argumentos del comando
#volumen
#nb boxes
#nb spaces
#volumenes de cajas
#L de cajas
#W de cajas
#H de cajas
#volumenes de espacios
#L de espacios
#W de espacios
#H de espacios
#volumen alcanzado (BSG 500s)

def read_txt(path):
    txt = open(path)
    data = (txt.read()).split('\n')
    txt.close()
    return data

def sample_list(data):
    sample_list = []
    for line in data:
        if ':' in line:
            sample = []
            sample_list.append(sample)
        if line != '':
            sample.append(line)
    return sample_list

def get_features(txt_sample_list):
    args = []
    state_volume = []
    nb_boxes = []
    nb_spaces = []
    boxes_volume = []
    l_boxes = []
    w_boxes = []
    h_boxes = []
    spaces_volume = []
    l_spaces = []
    w_spaces = []
    h_spaces = []
    volume_reached = []

    for file in txt_sample_list:
        for feature in file:
            if len(feature) == 13:
                args.append(feature[0])
                state_volume.append(float(feature[1]))
                nb_boxes.append(float(feature[2]))
                nb_spaces.append(float(feature[3]))
                boxes_volume.append(get_array_features(feature, 4))
                l_boxes.append(get_array_features(feature, 5))
                w_boxes.append(get_array_features(feature, 6))
                h_boxes.append(get_array_features(feature, 7))
                spaces_volume.append(get_array_features(feature, 8))
                l_spaces.append(get_array_features(feature, 9))
                w_spaces.append(get_array_features(feature, 10))
                h_spaces.append(get_array_features(feature, 11))
                volume_reached.append(float(feature[12]))
            else:
                print('Sample', feature[0].split(':')[0], 'has not been imported')
    
    return np.array(args), np.array(state_volume), np.array(nb_boxes), np.array(nb_spaces), np.array(boxes_volume), np.array(l_boxes), np.array(w_boxes), np.array(h_boxes), np.array(spaces_volume), np.array(l_spaces), np.array(w_spaces), np.array(h_spaces), np.array(volume_reached)

def get_array_features(feature, index):
    aux = np.array(feature[index].strip().split(' ')).astype('float32')
    aux = np.array([np.mean(aux), moment(aux, moment=2), moment(aux, moment=3), moment(aux, moment=4), moment(aux, moment=5)])
    return (aux - np.amin(aux)) / (np.amax(aux) - np.amin(aux)) if np.amin(aux) > 0 and np.amax(aux) > 0 else aux

def process_data(state_volume, nb_boxes, nb_spaces, boxes_volume, l_boxes, w_boxes, h_boxes, spaces_volume, l_spaces, w_spaces, h_spaces):
    data = []
    for file in range(0, len(boxes_volume)):
        data.append(np.concatenate((state_volume[file], nb_boxes[file], nb_spaces[file], boxes_volume[file], l_boxes[file], w_boxes[file], h_boxes[file], spaces_volume[file], l_spaces[file], w_spaces[file], h_spaces[file]), axis=None))
    return np.array(data)

## Neural Network Model

In [3]:
def export_test_metrics(first_it, last_it, list_test_metrics, random_state, name_metrics):
    output = ''
    for name in name_metrics:
        output += str(name) + '\t'
    output += '\n'
    for metrics in list_test_metrics:
        for m in metrics:
            output += str(m) + '\t'
        output += '\n'
    file = open('test_metrics/metrics' + str(first_it) + 'to' + str(last_it) + 'random_state' + str(random_state) + '.txt','w')
    file.write(output)
    file.close()
    print('Metrics save in:', os.getcwd())

In [4]:
def export_scores(first_it, last_it, list_scores, random_state):
    output = ''
    for score in list_scores:
        output = '{\nhidden layer:[' + str(score[0]) + ', ' + str(score[1]) + ']\nscore:' + str(score[2].history) + '\n}\n'
    file = open('scores/scores' + str(first_it) + 'to' + str(last_it) + 'random_state' + str(random_state) + '.json','w')
    file.write(output)
    file.close()
    print('Scores save in:', os.getcwd())

In [12]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def neural_network_model(histogram_data, volume_reached, first_it, last_it, random_state, epochs):
    
    print('Processed data shape:', processed_data.shape, end='\n\n')
    print('Volume reached shape:', volume_reached.shape, end='\n\n')
    
    for split in range(3, 4):#random_state + 1):
        x_train, x_test, y_train, y_test = train_test_split(histogram_data, volume_reached, test_size=0.25, random_state=split)

        print('x train shape:', x_train.shape)
        print('x test shape:', x_test.shape)
        print('y train shape:', y_train.shape)
        print('y test shape:', y_test.shape)
        print()

        scores = []
        test_metrics_list = []

        print('Starting Dense Neural Network')
        print('Start time:', time.asctime(time.localtime(time.time())))

        for i in range(18, last_it + 1):
            for j in range(first_it, 101):
                
                test_metrics = []
                
                print('Iteration: i =', i, 'j =', j)
                model = Sequential()
                model.add(Dense(i, input_dim=histogram_data.shape[1]))
                model.add(Activation('tanh'))
                model.add(Dense(j))
                model.add(Activation('tanh'))
                model.add(Dense(1))
                model.add(Activation('linear'))
                model.compile(loss='mae',
                              optimizer='adam',
                              metrics=['mse', root_mean_squared_error,'mae'])

                model.summary()

                scores.append([i, j, model.fit(x_train, y_train, epochs=epochs)])
                
                test_metrics.append(i)
                test_metrics.append(j)
                for tm in model.evaluate(x_test, y_test):
                    test_metrics.append(tm)
                test_metrics_list.append(test_metrics)
                del test_metrics
                
                if j % 25 == 0:
                    export_test_metrics(i, j, test_metrics_list, split, ['hidden_layer_0', 'hidden_layer_1', 'loss_mae', 'mse', 'root_mean_squared_error','mae'])
                    export_scores(i, j, scores, split)
                    del test_metrics_list
                    del scores
                    scores = []
                    test_metrics_list = []
        
                del model
                K.clear_session()

        '''export_test_metrics(first_it, last_it, test_metrics, random_state, ['hidden_layer_0', 'hidden_layer_1', 'loss_msre', 'accuracy', 'mse', 'root_mean_squared_error','mae'])
        export_scores(first_it, last_it, scores, random_state)'''

        '''del test_metrics
        del scores'''

        print('End time:', time.asctime(time.localtime(time.time())), end='\n\n')

## Script

### Import data

In [6]:
def export_processed_data(processed_data):
    output = ''
    for sample in processed_data:
        for feature in sample:
            output += str(feature) + '\t'
        output += '\n'
        
    file = open('processed_data' + str(len(processed_data)) + '.txt','w')
    file.write(output)
    file.close()
    print('Processed data save in:', os.getcwd())

In [7]:
with K.tf.device('/gpu:0'):
    DATADIR = "trainingset_allinfo/"
    txt_sample_list = []
    for name_txt in os.listdir(DATADIR):
        if ("trainingset_allinfo" in name_txt and ".txt" in name_txt) and '~$' not in name_txt:
            data = read_txt(DATADIR + name_txt)
            txt_sample_list.append(sample_list(data))

    args, state_volume, nb_boxes, nb_spaces, boxes_volume, l_boxes, w_boxes, h_boxes, spaces_volume, l_spaces, w_spaces, h_spaces, volume_reached = get_features(txt_sample_list)
    print('Imported data')
    processed_data = process_data(state_volume, nb_boxes, nb_spaces, boxes_volume, l_boxes, w_boxes, h_boxes, spaces_volume, l_spaces, w_spaces, h_spaces)
    print('Processed data')
    export_processed_data(processed_data)

Sample 15 has not been imported
Sample 24 has not been imported
Sample 29 has not been imported
Sample 32 has not been imported
Sample 45 has not been imported
Sample 50 has not been imported
Sample 52 has not been imported
Sample 61 has not been imported
Sample 71 has not been imported
Sample 90 has not been imported
Sample 95 has not been imported
Sample 99 has not been imported
Sample 110 has not been imported
Sample 147 has not been imported
Sample 155 has not been imported
Sample 157 has not been imported
Sample 158 has not been imported
Sample 164 has not been imported
Sample 177 has not been imported
Sample 178 has not been imported
Sample 183 has not been imported
Sample 196 has not been imported
Sample 202 has not been imported
Sample 208 has not been imported
Sample 217 has not been imported
Sample 225 has not been imported
Sample 226 has not been imported
Sample 234 has not been imported
Sample 243 has not been imported
Sample 247 has not been imported
Sample 261 has not bee

Sample 152 has not been imported
Sample 155 has not been imported
Sample 163 has not been imported
Sample 175 has not been imported
Sample 176 has not been imported
Sample 193 has not been imported
Sample 198 has not been imported
Sample 206 has not been imported
Sample 219 has not been imported
Sample 220 has not been imported
Sample 225 has not been imported
Sample 240 has not been imported
Sample 252 has not been imported
Sample 253 has not been imported
Sample 257 has not been imported
Sample 261 has not been imported
Sample 267 has not been imported
Sample 269 has not been imported
Sample 299 has not been imported
Sample 307 has not been imported
Sample 308 has not been imported
Sample 316 has not been imported
Sample 317 has not been imported
Sample 319 has not been imported
Sample 326 has not been imported
Sample 339 has not been imported
Sample 356 has not been imported
Sample 385 has not been imported
Sample 393 has not been imported
Sample 411 has not been imported
Sample 423

### Shape data

In [8]:
print(args.shape)
print(state_volume.shape)
print(nb_boxes.shape)
print(nb_spaces.shape)
print(boxes_volume.shape)
print(l_boxes.shape)
print(w_boxes.shape)
print(h_boxes.shape)
print(spaces_volume.shape)
print(l_spaces.shape)
print(w_spaces.shape)
print(h_spaces.shape)
print(volume_reached.shape)

(3745,)
(3745,)
(3745,)
(3745,)
(3745, 5)
(3745, 5)
(3745, 5)
(3745, 5)
(3745, 5)
(3745, 5)
(3745, 5)
(3745, 5)
(3745,)


### Start Neural Network Test

In [13]:
with K.tf.device('/gpu:0'):
    #neural_network_model(processed_data, volume_reached, 2, 20, 50)
    #neural_network_model(processed_data, volume_reached, 21, 40, 50)
    #neural_network_model(processed_data, volume_reached, 41, 60, 50)
    #neural_network_model(processed_data, volume_reached, 61, 80, 50)
    neural_network_model(processed_data, volume_reached, 2, 20, 5, 50)

Processed data shape: (3745, 43)

Volume reached shape: (3745,)

x train shape: (2808, 43)
x test shape: (937, 43)
y train shape: (2808,)
y test shape: (937,)

Starting Dense Neural Network
Start time: Fri Sep 13 22:42:12 2019
Iteration: i = 18 j = 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 18)                792       
_________________________________________________________________
activation_1 (Activation)    (None, 18)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 38        
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
_______________________

2808/2808 [==============================] - 0s 72us/step - loss: 0.0095 - mean_squared_error: 2.6279e-04 - root_mean_squared_error: 0.0095 - mean_absolute_error: 0.0095
Epoch 39/50
2808/2808 [==============================] - 0s 75us/step - loss: 0.0089 - mean_squared_error: 2.3772e-04 - root_mean_squared_error: 0.0089 - mean_absolute_error: 0.0089
Epoch 40/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0095 - mean_squared_error: 2.5264e-04 - root_mean_squared_error: 0.0095 - mean_absolute_error: 0.0095
Epoch 41/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0093 - mean_squared_error: 2.4761e-04 - root_mean_squared_error: 0.0093 - mean_absolute_error: 0.0093
Epoch 42/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0092 - mean_squared_error: 2.4044e-04 - root_mean_squared_error: 0.0092 - mean_absolute_error: 0.0092
Epoch 43/50
2808/2808 [==============================] - 0s 73us/step - loss: 0.0091 - mean_squared_er

2808/2808 [==============================] - 0s 78us/step - loss: 0.0102 - mean_squared_error: 2.8443e-04 - root_mean_squared_error: 0.0102 - mean_absolute_error: 0.0102
Epoch 27/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0098 - mean_squared_error: 2.6941e-04 - root_mean_squared_error: 0.0098 - mean_absolute_error: 0.0098
Epoch 28/50
2808/2808 [==============================] - 0s 68us/step - loss: 0.0094 - mean_squared_error: 2.6672e-04 - root_mean_squared_error: 0.0094 - mean_absolute_error: 0.0094
Epoch 29/50
2808/2808 [==============================] - 0s 74us/step - loss: 0.0098 - mean_squared_error: 2.6661e-04 - root_mean_squared_error: 0.0098 - mean_absolute_error: 0.0098
Epoch 30/50
2808/2808 [==============================] - 0s 74us/step - loss: 0.0093 - mean_squared_error: 2.4253e-04 - root_mean_squared_error: 0.0093 - mean_absolute_error: 0.0093
Epoch 31/50
2808/2808 [==============================] - 0s 86us/step - loss: 0.0089 - mean_squared_er

2808/2808 [==============================] - 0s 70us/step - loss: 0.0096 - mean_squared_error: 2.6466e-04 - root_mean_squared_error: 0.0096 - mean_absolute_error: 0.0096
Epoch 15/50
2808/2808 [==============================] - 0s 69us/step - loss: 0.0094 - mean_squared_error: 2.5500e-04 - root_mean_squared_error: 0.0094 - mean_absolute_error: 0.0094
Epoch 16/50
2808/2808 [==============================] - 0s 69us/step - loss: 0.0087 - mean_squared_error: 2.3508e-04 - root_mean_squared_error: 0.0087 - mean_absolute_error: 0.0087
Epoch 17/50
2808/2808 [==============================] - 0s 69us/step - loss: 0.0088 - mean_squared_error: 2.2943e-04 - root_mean_squared_error: 0.0088 - mean_absolute_error: 0.0088
Epoch 18/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0089 - mean_squared_error: 2.3003e-04 - root_mean_squared_error: 0.0089 - mean_absolute_error: 0.0089
Epoch 19/50
2808/2808 [==============================] - 0s 68us/step - loss: 0.0085 - mean_squared_er

2808/2808 [==============================] - 0s 72us/step - loss: 0.0175 - mean_squared_error: 8.8554e-04 - root_mean_squared_error: 0.0175 - mean_absolute_error: 0.0175
Epoch 3/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0135 - mean_squared_error: 5.8085e-04 - root_mean_squared_error: 0.0135 - mean_absolute_error: 0.0135
Epoch 4/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0120 - mean_squared_error: 4.2805e-04 - root_mean_squared_error: 0.0120 - mean_absolute_error: 0.0120
Epoch 5/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0111 - mean_squared_error: 3.4853e-04 - root_mean_squared_error: 0.0111 - mean_absolute_error: 0.0111
Epoch 6/50
2808/2808 [==============================] - 0s 73us/step - loss: 0.0109 - mean_squared_error: 3.4365e-04 - root_mean_squared_error: 0.0109 - mean_absolute_error: 0.0109
Epoch 7/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0111 - mean_squared_error: 

...

2808/2808 [==============================] - 0s 72us/step - loss: 0.0115 - mean_squared_error: 2.8205e-04 - root_mean_squared_error: 0.0115 - mean_absolute_error: 0.0115
Epoch 11/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0189 - mean_squared_error: 5.7578e-04 - root_mean_squared_error: 0.0189 - mean_absolute_error: 0.0189
Epoch 12/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0136 - mean_squared_error: 3.4220e-04 - root_mean_squared_error: 0.0136 - mean_absolute_error: 0.0136
Epoch 13/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0111 - mean_squared_error: 2.7090e-04 - root_mean_squared_error: 0.0111 - mean_absolute_error: 0.0111
Epoch 14/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0125 - mean_squared_error: 3.0944e-04 - root_mean_squared_error: 0.0125 - mean_absolute_error: 0.0125
Epoch 15/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0158 - mean_squared_er

Epoch 1/50
2808/2808 [==============================] - 0s 131us/step - loss: 0.0714 - mean_squared_error: 0.0178 - root_mean_squared_error: 0.0714 - mean_absolute_error: 0.0714
Epoch 2/50
2808/2808 [==============================] - 0s 75us/step - loss: 0.0299 - mean_squared_error: 0.0015 - root_mean_squared_error: 0.0299 - mean_absolute_error: 0.0299
Epoch 3/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0201 - mean_squared_error: 8.4938e-04 - root_mean_squared_error: 0.0201 - mean_absolute_error: 0.0201
Epoch 4/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0185 - mean_squared_error: 6.7154e-04 - root_mean_squared_error: 0.0185 - mean_absolute_error: 0.0185
Epoch 5/50
2808/2808 [==============================] - 0s 69us/step - loss: 0.0143 - mean_squared_error: 4.5402e-04 - root_mean_squared_error: 0.0143 - mean_absolute_error: 0.0143
Epoch 6/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0241 - mean_squared_err

2808/2808 [==============================] - 0s 71us/step - loss: 0.0134 - mean_squared_error: 3.4958e-04 - root_mean_squared_error: 0.0134 - mean_absolute_error: 0.0134
Epoch 47/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0132 - mean_squared_error: 3.3657e-04 - root_mean_squared_error: 0.0132 - mean_absolute_error: 0.0132
Epoch 48/50
2808/2808 [==============================] - 0s 74us/step - loss: 0.0181 - mean_squared_error: 5.4514e-04 - root_mean_squared_error: 0.0181 - mean_absolute_error: 0.0181
Epoch 49/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0124 - mean_squared_error: 3.2163e-04 - root_mean_squared_error: 0.0124 - mean_absolute_error: 0.0124
Epoch 50/50
937/937 [==============================] - 0s 54us/step
Iteration: i = 20 j = 99
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                880 

2808/2808 [==============================] - 0s 72us/step - loss: 0.0155 - mean_squared_error: 4.3972e-04 - root_mean_squared_error: 0.0155 - mean_absolute_error: 0.0155
Epoch 35/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0143 - mean_squared_error: 4.3454e-04 - root_mean_squared_error: 0.0143 - mean_absolute_error: 0.0143
Epoch 36/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0118 - mean_squared_error: 2.9555e-04 - root_mean_squared_error: 0.0118 - mean_absolute_error: 0.0118
Epoch 37/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0131 - mean_squared_error: 3.4383e-04 - root_mean_squared_error: 0.0131 - mean_absolute_error: 0.0131
Epoch 38/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0117 - mean_squared_error: 2.8963e-04 - root_mean_squared_error: 0.0117 - mean_absolute_error: 0.0117
Epoch 39/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0124 - mean_squared_er

2808/2808 [==============================] - 0s 71us/step - loss: 0.0135 - mean_squared_error: 3.6358e-04 - root_mean_squared_error: 0.0135 - mean_absolute_error: 0.0135
Epoch 23/50
2808/2808 [==============================] - 0s 72us/step - loss: 0.0117 - mean_squared_error: 2.9037e-04 - root_mean_squared_error: 0.0117 - mean_absolute_error: 0.0117
Epoch 24/50
2808/2808 [==============================] - 0s 68us/step - loss: 0.0188 - mean_squared_error: 5.8838e-04 - root_mean_squared_error: 0.0188 - mean_absolute_error: 0.0188
Epoch 25/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0119 - mean_squared_error: 3.0234e-04 - root_mean_squared_error: 0.0119 - mean_absolute_error: 0.0119
Epoch 26/50
2808/2808 [==============================] - 0s 71us/step - loss: 0.0124 - mean_squared_error: 3.0610e-04 - root_mean_squared_error: 0.0124 - mean_absolute_error: 0.0124
Epoch 27/50
2808/2808 [==============================] - 0s 70us/step - loss: 0.0133 - mean_squared_er

In [ ]:
#print(scores[0].history.keys())
# "Loss"
#plt.plot(history.history['loss'])
#plt.plot(history.history['acc'])
#plt.plot(history.history['mean_squared_error'])
#plt.plot(scores[0].history['root_mean_squared_error'])
#plt.title('model loss')
#plt.ylabel('root_mean_squared_error')
#plt.xlabel('epoch')
#plt.legend(['root_mean_squared_error'], loc='upper left')
#plt.show()

### Start Neural Network with parameters

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(processed_data, volume_reached, test_size=0.25, random_state=1)

#print(x_train.shape)
#print(x_test.shape)
#print(y_train.shape)
#print(y_test.shape)

epochs = 69

print('Starting Dense Neural Network')
print('Start time:', time.asctime(time.localtime(time.time())))

model = Sequential()
model.add(Dense(86, input_dim=processed_data.shape[1]))
model.add(Activation('sigmoid'))
model.add(Dense(60))
model.add(Activation('sigmoid'))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss='mae',
              optimizer='adam',
              metrics=['accuracy', 'mse', root_mean_squared_error,'mae'])

model.summary()

scores = model.fit(x_train, y_train, epochs=epochs)
test_metrics = model.evaluate(x_test, y_test)

del model
K.clear_session()

#export_test_metrics(first_it, last_it, test_metrics, ['loss_msre', 'accuracy', 'mse', 'root_mean_squared_error','mae'])
#export_scores(first_it, last_it, scores)
print('End time:', time.asctime(time.localtime(time.time())), end='\n\n')
print('loss:', test_metrics[0])
print('accuracy:', test_metrics[1])
print('mse:', test_metrics[2])
print('rmse:', test_metrics[3])
print('mae:', test_metrics[4])

In [ ]:
from scipy.stats import kurtosis
import numpy as np

#Format data:
#argumentos del comando
#volumen
#nb boxes
#nb spaces
#volumenes de cajas
#L de cajas
#W de cajas
#H de cajas
#volumenes de espacios
#L de espacios
#W de espacios
#H de espacios
#volumen alcanzado (BSG 500s)

a = [0.00652783, 0.00652783, 0.00652783, 0.00652783, 0.00652783, 0.00652783, 0.00652783, 0.00652783, 0.00134, 0.00134, 0.00134, 0.00134, 0.00134, 0.00134, 0.0131466, 0.0131466, 0.00807488, 0.00807488, 0.00356508, 0.00356508, 0.00356508, 0.00356508, 0.00356508, 0.00356508, 0.00286052, 0.00286052, 0.00286052, 0.00286052, 0.00286052, 0.00286052, 0.0148113, 0.0148113, 0.0148113, 0.0148113, 0.0148113, 0.0148113, 0.0148113, 0.00509199, 0.00509199, 0.00509199, 0.00509199, 0.00509199, 0.00509199, 0.00509199, 0.00509199, 0.0183822, 0.0183822, 0.0183822, 0.0183822, 0.0183822, 0.0183822, 0.0183822, 0.0183822, 0.0183822, 0.0136456, 0.0136456, 0.0136456, 0.0136456, 0.0136456, 0.0136456, 0.0136456, 0.0136456, 0.00363235, 0.00363235, 0.00363235, 0.00363235, 0.00363235, 0.00363235, 0.00363235, 0.018735]

print(kurtosis(a))

In [ ]:
a1 = np.sum(np.array(a))/len(a)
a2 = np.sum(np.array(a)**2)/len(a)
a3 = np.sum(np.array(a)**3)/len(a)
a4 = np.sum(np.array(a)**4)/len(a)

m1 = a1 - a1
m2 = a2 - (a1**2)
m3 = a3 - 3*a1*a2 + 2*a1**3
m4 = a4 - 4*a1*a3 + 6*a1**2*a2 - 3*a1**4
print(a1)
print(a2)
print(a3)
print(m4)